In [2]:
import pandas as pd
import numpy as np
import sys, os

In [31]:
#Pre-process the dataset
input_path='/fastscratch/c-panz/2021-09-13/test'
df_T4=pd.read_csv(os.path.join(input_path, 'T4.test.bed'), sep='\t')
df_lambda=pd.read_csv(os.path.join(input_path, 'lambda.test.bed'), sep='\t')
df_5mClambda=pd.read_csv(os.path.join(input_path, 'lambda_5mC.test.bed'), sep='\t')

In [32]:
# Add the label column
df_T4['label'] = '5hmC'
df_lambda['label'] = '5C'
df_5mClambda['label'] = '5mC'
# Merge the dataset 
df = pd.concat([df_T4, df_5mClambda, df_lambda])

# Create label column
# Createa a label with representations: 5C = 0, 5mC = 1, 5hmC = 2
df['label'] = df['label'].map({'5C':0, '5mC':1, '5hmC':2 })
df.to_csv(os.path.join(input_path, 'total.test.bed'), sep='\t', index = None)

In [ ]:

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

In [29]:
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from statistics import mean

# https://medium.com/sfu-cspmp/surviving-in-a-random-forest-with-imbalanced-datasets-b98b963d52eb
# Build the random forest model
example_params = {
        'n_estimators': 20,
        'max_depth': 5,
        'random_state': 42
    }

rf_model = RandomForestClassifier(**example_params)

#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#Randomly spilt dataset to traing/testing dataset with the original ratio
X_train, X_test, y_train, y_test = train_test_split(df_feature, 
                                                    df_class, 
                                                    test_size=0.2, 
                                                    stratify=df_class)


#https://medium.com/sfu-cspmp/surviving-in-a-random-forest-with-imbalanced-datasets-b98b963d52eb
scoring = ('accuracy', 'recall_macro', 'f1_macro')

#Evaluate SRF model
scores = cross_validate(rf_model, X_train, y_train, scoring=scoring, cv=cv)
#Get average evaluation metrics
print('Mean accuracy: %.3f' % mean(scores['test_accuracy']))
print('Mean recall_macro: %.3f' % mean(scores['test_recall_macro']))
print('Mean f1_macro: %.3f' % mean(scores['test_f1_macro']))

Mean accuracy: 0.931
Mean recall_macro: 0.931
Mean f1_macro: 0.931


In [30]:
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from statistics import mean

# https://medium.com/sfu-cspmp/surviving-in-a-random-forest-with-imbalanced-datasets-b98b963d52eb
# Build the random forest model
example_params = {
        'n_estimators': 20,
        'max_depth': 5,
        'random_state': 42
    }

rf_model = RandomForestClassifier(**example_params)

#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#Randomly spilt dataset to traing/testing dataset with the original ratio
X_train, X_test, y_train, y_test = train_test_split(df_feature, 
                                                    df_class, 
                                                    test_size=0.2, 
                                                    stratify=df_class)


#https://medium.com/sfu-cspmp/surviving-in-a-random-forest-with-imbalanced-datasets-b98b963d52eb
scoring = ('accuracy', 'recall_macro', 'f1_macro')


cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='recall_macro', n_jobs=-1)
print('test4')

test4
